# 🚖 FleetPy Demo Notebook

Welcome to FleetPy! This notebook will guide you through setting up and running your first fleet simulation using FleetPy, an open-source framework for modeling and controlling vehicle fleets in ride-sharing applications.

## 🎯 What we'll cover:
1. 🛠 Environment Setup
2. 📦 Package Installation
3. 🗺 Loading Example Data
4. 🚀 Running a Basic Simulation
5. 📊 Visualizing Results

## 🛠 Environment Setup

First, let's make sure we have all required packages installed. Run the following cell to check your Python environment and install any missing dependencies.

In [1]:
# Check Python version and install required packages
import sys
print(f"Python version: {sys.version}")

# Import required packages
try:
    import numpy as np
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objects as go
    import geopandas as gpd
    from utils.analysis import *
    print("✅ All required packages are installed!")
except ImportError as e:
    print(f"❌ Missing package: {str(e)}")
    print("\nPlease run: conda env create -f environment.yml")

Python version: 3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:23:34) [Clang 18.1.8 ]
✅ All required packages are installed!


## 📦 Import FleetPy

Now that we have our environment set up, let's import the necessary FleetPy modules and set up a basic simulation using the example data.

In [2]:
import os
import sys

# Add FleetPy to path if needed
fleetpy_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if fleetpy_path not in sys.path:
    sys.path.append(fleetpy_path)

# Import FleetPy modules
from src.FleetSimulationBase import FleetSimulationBase
from src.misc.globals import *

print("✅ FleetPy modules imported successfully!")

✅ FleetPy modules imported successfully!


## 🚀 Setting up the Simulation

In FleetPy, simulations are configured using two CSV files:
1. A constant configuration file (`constant_config_*.csv`) that defines study-wide parameters
2. A scenario configuration file (`example_*.csv`) that defines scenario-specific parameters

We'll use the basic pooling example with ImmediateOfferEnvironment:
- Constant config: `studies/example_study/scenarios/constant_config_ir.csv`
- Scenario config: `studies/example_study/scenarios/example_ir_only.csv`

## 🎛️ Simulation Configuration Walkthrough

Before loading the example configs, let's understand the key parameters we can configure in FleetPy. We'll create our own configuration file with the essential parameters:

1. **Network and Demand**
   - `network_name`: `example_network`
   - `demand_name`: `example_demand`
   - `zone_system_name`: `example_zones`

2. **Time Settings**
   - `time_step`: `30` (seconds)
   - `start_time`: `0` (seconds from midnight)
   - `end_time`: `3600` (seconds from midnight)

3. **Fleet Settings**
   - `nr_mod_operators`: `1` (number of operators)
   - `op_fleet_composition`: `default_vehtype:10` (fleet size and type)

4. **Operational Settings**
   - `sim_env`: `ImmediateDecisionsSimulation`
   - `user_max_decision_time`: `0`
   - `rq_type`: `BasicRequest`
   - `network_type`: `NetworkBasicWithStore`
   - `op_max_wait_time`: `300`
   - `op_max_detour_time_factor`: `1.4`
   - `op_const_boarding_time`: `30`
   - `op_vr_control_func_dict`: `func_key:distance_and_user_times_with_walk;vot:0.45`
   - `n_cpu_per_sim`: `1`
   - `log_level`: `info`
   - `random_seed`: `0`

Let's create and save our custom configuration!

In [3]:
import pandas as pd
import os

# Create a dictionary with the essential parameters using global variable names
config_params = {
    # Simulation environment
    # Use immediate decisions simulation environment
    G_SIM_ENV: 'ImmediateDecisionsSimulation',

    # No max decision time for immediate decisions
    G_AR_MAX_DEC_T: 0,
    # Basic request type for testing. Always accepts the operator's offer
    G_RQ_TYP1: 'BasicRequest',

    # Network and Demand
    G_NETWORK_NAME: 'example_network',             # Basic network for testing
    G_DEMAND_NAME: 'example_demand',               # Example demand pattern
    G_ZONE_SYSTEM_NAME: 'example_zones',           # Service area zones

    # Time Settings
    G_SIM_TIME_STEP: 30,                           # Update every 30 seconds
    G_SIM_START_TIME: 0,                           # Start at midnight
    G_SIM_END_TIME: 3600,                          # Run for 1 hour

    # Operational Settings
    G_NETWORK_TYPE: 'NetworkBasicWithStore',       # Basic network with store
    G_OP_MAX_WT: 300,                              # Max wait time of 5 minutes
    G_OP_MAX_DTF: 1.4,                             # Allow 40% detour
    # Constant boarding time of 30 seconds
    G_OP_CONST_BT: 30,
    G_NR_OPERATORS: 1,                             # Number of operators
    # Value of time function for vehicle routing control
    G_OP_VR_CTRL_F: 'func_key:distance_and_user_times_with_walk;vot:0.45',

    # Simulation Settings
    G_SLAVE_CPU: 1,                                # Use 1 CPU for slave processes
    "log_level": "info",                           # Set log level to INFO
    # Use a fixed random seed for reproducibility
    G_RANDOM_SEED: 0
}

# Convert to DataFrame for CSV format
config_df = pd.DataFrame(list(config_params.items()),
                         columns=['Parameter', 'Value'])

# Save constant config
config_dir = 'scenarios'
os.makedirs(config_dir, exist_ok=True)
constant_config_path = os.path.join(config_dir, 'custom_constant_config.csv')
config_df.to_csv(constant_config_path, index=False)

# Create a minimal scenario config (can override constant config values)
scenario_params = {
    G_STUDY_NAME: 'example_study',                 # Name of the study
    G_SCENARIO_NAME: 'example_pool_irsonly_sc_1',  # Scenario name
    G_RQ_FILE: "example_100.csv",                  # Example request file
    G_OP_FLEET: "default_vehtype:10",              # Size of the fleet
    # Operational module for pooling with IRS
    G_OP_MODULE: 'PoolingIRSOnly',
}
scenario_df = pd.DataFrame([scenario_params])
scenario_path = os.path.join(config_dir, 'scenario_custom_config.csv')
scenario_df.to_csv(scenario_path, index=False)

print("✅ Custom configuration files created!")
print("\n📝 Current configuration:")
for param, value in config_params.items():
    print(f"  - {param:.<30} {value}")

✅ Custom configuration files created!

📝 Current configuration:
  - sim_env....................... ImmediateDecisionsSimulation
  - user_max_decision_time........ 0
  - rq_type....................... BasicRequest
  - network_name.................. example_network
  - demand_name................... example_demand
  - zone_system_name.............. example_zones
  - time_step..................... 30
  - start_time.................... 0
  - end_time...................... 3600
  - network_type.................. NetworkBasicWithStore
  - op_max_wait_time.............. 300
  - op_max_detour_time_factor..... 1.4
  - op_const_boarding_time........ 30
  - nr_mod_operators.............. 1
  - op_vr_control_func_dict....... func_key:distance_and_user_times_with_walk;vot:0.45
  - n_cpu_per_sim................. 1
  - log_level..................... info
  - random_seed................... 0


### 🔄 Try Different Values!

You can experiment with different parameter values to see how they affect the simulation:
- Increase/decrease `op_fleet_composition` to see impact on service rate
- Modify `op_max_detour_time_factor` to allow more/less pooling
- Change simulation duration with `end_time`
- Adjust `time_step` for different update frequencies

Just modify the values in the `config_params` dictionary above and run the cell again to create new config files.

In [4]:
# Load and initialize simulation with our custom configs
import os
import src.misc.config as config
from src.misc.init_modules import load_simulation_environment

# Set up paths to our custom config files
scs_path = os.path.join(os.getcwd(), 'scenarios')
constant_config_file = os.path.join(scs_path, 'custom_constant_config.csv')
scenario_file = os.path.join(scs_path, 'scenario_custom_config.csv')

# Read configuration files
constant_cfg = config.ConstantConfig(constant_config_file)
scenario_cfgs = config.ScenarioConfig(scenario_file)

# Combine configurations
scenario_cfg = constant_cfg + scenario_cfgs[0]

# Initialize simulation
sim = load_simulation_environment(scenario_cfg)

# Run the simulation
sim.run()

--------------------------------------------------------------------------------
Simulation of scenario example_pool_irsonly_sc_1
Only minimum output to console -> see log-file
Scenario example_pool_irsonly_sc_1
Operator 0 control strategy: PoolingInsertionHeuristicOnly
	 control function: {'func_key': 'distance_and_user_times_with_walk', 'vot': 0.45}
Operator 0 additional strategies:
	 RV Heuristics: {}
	 Stop-Insert Heuristics: {}
	 Charging: None
	 On-Street Parking: True
	 Repositioning: None
	 Dynamic Pricing: None
	 Dynamic Fleet Sizing: None



example_pool_irsonly_sc_1: 100%|██████████| 100/100 [00:01<00:00, 50.96it/s, simulation_time=3570, driving=4, idle=6, charging=0, reposition=0]


Scenario example_pool_irsonly_sc_1 finished:
      initialization : 0:00:00 h
          simulation : 0:00:01 h
          evaluation : 0:00:00 h



## 📊 Results Analysis

We'll use our analysis module to visualize and analyze the simulation results. The analysis is split into the following updated sections:
1. **Key Performance Indicators (KPIs)**
2. **User Experience Analysis**
3. **Travel Time Analysis**
4. **Vehicle Status Distribution**
5. **Individual Vehicle Performance**

### 📊 Key Performance Indicators (KPIs)

Let's start by examining the overall performance metrics of our simulation. These KPIs provide a high-level view of how well our fleet is serving customer demands, including metrics like service rate, wait times, and vehicle utilization.

In [5]:
# Define the results directory based on the scenario name
results_dir = os.path.join(os.getcwd(), 'results',
                           scenario_cfg[G_SCENARIO_NAME])

In [6]:
# Get KPI summary
kpi_summary = analyze_kpis(results_dir)

# Display KPIs
print('📈 Simulation KPIs:')
print(kpi_summary.to_string(index=False))

📈 Simulation KPIs:
                 Metric      Value
         Total Requests  48.000000
        Served Requests  48.000000
Average Wait Time (min)   1.927324
Average Trip Time (min)   3.074630
    Total Distance (km) 119.855174
    Empty Distance (km)  44.103579
       Service Rate (%) 100.000000
     Created Offers (%) 100.000000
  Fleet Utilization (%)  39.904432
     Occupancy Rate (%)   0.632026


### 👥 User Experience Analysis

Understanding the user experience is crucial for ride-sharing services. Here we analyze wait times and service quality from the customer's perspective. The visualization shows the distribution of wait times and helps identify any service bottlenecks.

In [7]:
# Analyze user statistics
fig, wait_time_stats = analyze_user_stats(results_dir)
fig.show()

print('\n⏱️ Wait Time Statistics (minutes):')
print(wait_time_stats.to_string())


⏱️ Wait Time Statistics (minutes):
count    0.800000
mean     0.032122
std      0.023121
min      0.000000
25%      0.010205
50%      0.030377
75%      0.049359
max      0.077168


### 🎯 Travel Time Analysis

Let's dive deeper into the travel time efficiency by comparing actual travel times with direct route times. This helps us understand how well the system minimizes detours and maintains efficient routing.

In [8]:
# Analyze detailed user experience metrics
fig, user_stats = analyze_detailed_user_experience(results_dir)
fig.show()

print("\n🕒 Travel Time Analysis:")
print(f"Average detour ratio: {user_stats['Travel Time Efficiency']['mean_ratio']:.2f}")
print(f"Median detour ratio: {user_stats['Travel Time Efficiency']['median_ratio']:.2f}")
print(f"Standard deviation: {user_stats['Travel Time Efficiency']['std_ratio']:.2f}")


🕒 Travel Time Analysis:
Average detour ratio: 1.34
Median detour ratio: 1.21
Standard deviation: 0.48


### 🚗 Vehicle Status Distribution

This analysis shows how vehicles spend their time during the simulation. Understanding the distribution between states (idle, pickup, occupied, etc.) helps optimize fleet size and identify potential inefficiencies in vehicle utilization.

In [9]:
# Analyze vehicle status distribution
fig = analyze_vehicle_status(results_dir)

fig.show()

### 🎯 Individual Vehicle Performance

Finally, we'll dive into vehicle-specific metrics to understand how individual vehicles in the fleet are performing. This helps identify any imbalances in vehicle utilization and areas for optimization in vehicle assignment strategies.

In [10]:
# Analyze vehicle-specific performance
fig, performance_stats = analyze_vehicle_performance(results_dir)
fig.show()

print('\n🚕 Vehicle Performance Statistics:')
for metric, stats in performance_stats.items():
    print(f'\n{metric}:')
    print(stats.to_string())


🚕 Vehicle Performance Statistics:

total km:
count    10.00
mean     11.99
std       5.18
min       1.27
25%      11.17
50%      13.15
75%      15.30
max      18.50

total costs:
count      10.00
mean     2799.70
std       129.56
min      2532.00
25%      2779.75
50%      2829.00
75%      2882.25
max      2963.00

total CO2 [g]:
count    10.00
mean      6.71
std       2.90
min       0.71
25%       6.26
50%       7.37
75%       8.57
max      10.36


## 📝 Conclusion
- 🚦 **Setup & Configuration:** We showed how to set up FleetPy, create custom configuration files, and initialize a simulation environment.
- 📊 **Performance Analysis:** Explored key performance indicators (KPIs) such as service rate, wait times, and fleet utilization to assess system effectiveness.
- 👥 **User Experience:** Analyzed user wait times and travel efficiency to understand service quality from the passenger perspective.
- 🚗 **Vehicle Metrics:** Examined vehicle status distribution and individual vehicle performance to identify utilization patterns and optimization opportunities.
- 🔧 **Experimentation:** Encouraged experimenting with configuration parameters (like fleet size or detour factors) to see their impact on results.
- 🧩 **Flexibility:** FleetPy enables you to simulate, analyze, and optimize a wide range of shared mobility scenarios—try out your own ideas and analyses!

Thank you for exploring FleetPy—happy simulating!